In [1]:
from src.route.service.module.utils import const
path = const.Path()
path.rembg_temp

'/Users/sakura0moti/Documents/git/sharemoti3/api/arhcive/revmg_temp'

In [3]:
import sqlite3
dbname = '/Volumes/share/server/git/sharemotiApi2/main/share.db'
conn = sqlite3.connect(dbname)
# sqliteを操作するカーソルオブジェクトを作成
cur = conn.cursor()

# wordlistというtableを作成してみる
# 大文字部はSQL文。小文字でも問題ない。
cur.execute(
    """Delete from wordList2 where desc like '%"%'
            """
)

# データベースへコミット。これで変更が反映される。
conn.commit()
conn.close()